<a href="https://colab.research.google.com/github/ptx-pluto/colab-notebooks/blob/master/rnw_sympy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sympy
sympy.init_printing()

In [ ]:
def rot_x(rad):
  c = sympy.cos(rad)
  s = sympy.sin(rad)
  return sympy.Matrix([[1,0, 0],
                       [0,c,-s],
                       [0,s, c]])

def rot_y(rad):
  c = sympy.cos(rad)
  s = sympy.sin(rad)  
  return sympy.Matrix([[ c,0,s],
                       [ 0,1,0],
                       [-s,0,c]])

def rot_z(rad):
  c = sympy.cos(rad)
  s = sympy.sin(rad)
  return sympy.Matrix([[c,-s,0],
                       [s, c,0],
                       [0, 0,1]])

`linear_decomp` function decompose the expression `expr` into linear combination of symbols `syms`

$V_{result} \cdot V_{syms} = S_{expr}$ 

$V$ for vector, $S$ for scalar

It is used when we convert equations into matrix form.

In [ ]:
def linear_decomp(expr,syms):
  def calc_term(i):
    return expr.subs([(syms[j],sympy.S.One) if i==j else (syms[j],0) for j in range(len(syms))])
  return [ calc_term(i) for i in range(len(syms))]

In [ ]:
cone = {
    'm': sympy.symbols('m'),
    'g': sympy.symbols('g'),
    'r': sympy.symbols('r'),
    'xCM': sympy.symbols('xCM'),
    'zCM': sympy.symbols('zCM'),
    'xCP': sympy.symbols('xCP'),
    'zCP': sympy.symbols('zCP'),
}

### Coordinate System

5 Generalized Coordiates: $q_1, q_2, q_4, q_5, q_6$

$q_1 = x$

$q_2 = y$

$q_3 = \psi$ (precession)

$q_4 = \theta$ (nutation)

$q_5 = \phi$ (spin)

In [ ]:
t = sympy.symbols('t')

q1 = sympy.Function('q_1')(t)
q2 = sympy.Function('q_2')(t)
q3 = sympy.Function('q_3')(t)
q4 = sympy.Function('q_4')(t)
q5 = sympy.Function('q_5')(t)

dq1 = sympy.Function('\dot{q}_1')(t)
dq2 = sympy.Function('\dot{q}_2')(t)
dq3 = sympy.Function('\dot{q}_3')(t)
dq4 = sympy.Function('\dot{q}_4')(t)
dq5 = sympy.Function('\dot{q}_5')(t)

psi   = q3
theta = q4
phi   = q5

psi_dot   = dq3
theta_dot = dq4
phi_dot   = dq5

ddq1, ddq2, ddq3, ddq4, ddq5 = sympy.symbols('\ddot{q}_1 \ddot{q}_2 \ddot{q}_3 \ddot{q}_4 \ddpt{q}_5')

q = (q1, q2, q3, q4, q5)
dq = (dq1, dq2, dq3, dq4, dq5)
ddq = (ddq1, ddq2, ddq3, ddq4, ddq5)

Apply the following subtitution rule after calculating time derivative of an expression.

In [ ]:
sub_rules = [(q1.diff(),dq1),(dq1.diff(),ddq1),
             (q2.diff(),dq2),(dq2.diff(),ddq2),
             (q3.diff(),dq3),(dq3.diff(),ddq3),
             (q4.diff(),dq4),(dq4.diff(),ddq4),
             (q5.diff(),dq5),(dq5.diff(),ddq5)]

### Rigid Body Transformation

$\vec{P}_{W} = R_{WB} \vec{P}_B + T_{WB}$

$R_{WB} = R_z(\psi) R_y(\theta) R_z(\phi)$

$T_{WB} = (\ q_1, q_2, \mathbf{r} sin(q_5) \ )^T$

In [ ]:
R_wb = rot_z(psi) * rot_y(theta) * rot_z(phi)
T_wb = sympy.Matrix([ q1, q2, cone['r'] * sympy.sin(theta)])
R_wb, T_wb

⎛⎡-sin(q₃(t))⋅sin(q₅(t)) + cos(q₃(t))⋅cos(q₄(t))⋅cos(q₅(t))  -sin(q₃(t))⋅cos(q
⎜⎢                                                                            
⎜⎢sin(q₃(t))⋅cos(q₄(t))⋅cos(q₅(t)) + sin(q₅(t))⋅cos(q₃(t))   -sin(q₃(t))⋅sin(q
⎜⎢                                                                            
⎝⎣                 -sin(q₄(t))⋅cos(q₅(t))                                     

₅(t)) - sin(q₅(t))⋅cos(q₃(t))⋅cos(q₄(t))  sin(q₄(t))⋅cos(q₃(t))⎤  ⎡   q₁(t)   
                                                               ⎥  ⎢           
₅(t))⋅cos(q₄(t)) + cos(q₃(t))⋅cos(q₅(t))  sin(q₃(t))⋅sin(q₄(t))⎥, ⎢   q₂(t)   
                                                               ⎥  ⎢           
 sin(q₄(t))⋅sin(q₅(t))                         cos(q₄(t))      ⎦  ⎣r⋅sin(q₄(t)

 ⎤⎞
 ⎥⎟
 ⎥⎟
 ⎥⎟
)⎦⎠

`CoM_b` is center of mass in body frame

In [ ]:
CoM_b = sympy.Matrix([ cone['xCM'], 0, cone['zCM'] ])
CoM_b

⎡xCM⎤
⎢   ⎥
⎢ 0 ⎥
⎢   ⎥
⎣zCM⎦

`CoM_w` is center of mass in world frame

In [ ]:
CoM_w = R_wb * CoM_b + T_wb
CoM_w

⎡xCM⋅(-sin(q₃(t))⋅sin(q₅(t)) + cos(q₃(t))⋅cos(q₄(t))⋅cos(q₅(t))) + zCM⋅sin(q₄(
⎢                                                                             
⎢xCM⋅(sin(q₃(t))⋅cos(q₄(t))⋅cos(q₅(t)) + sin(q₅(t))⋅cos(q₃(t))) + zCM⋅sin(q₃(t
⎢                                                                             
⎣                     r⋅sin(q₄(t)) - xCM⋅sin(q₄(t))⋅cos(q₅(t)) + zCM⋅cos(q₄(t)

t))⋅cos(q₃(t)) + q₁(t)⎤
                      ⎥
))⋅sin(q₄(t)) + q₂(t) ⎥
                      ⎥
)                     ⎦

### Equations of Motion from Lagarangian

$$
T = \frac{1}{2} m v_{CM}^2 \quad V = mgz_{CM} \quad L = T-V
$$

$$
\frac{\partial}{dt}\frac{\partial L}{\partial \dot{q}_i} - \frac{\partial L}{\partial q_i} = 0
$$

In [ ]:
CM_vel = sympy.simplify(CoM_w.diff(t).subs(sub_rules))
CM_vel

⎡-xCM⋅(\dot{q}₃(t)⋅sin(q₃(t))⋅cos(q₄(t))⋅cos(q₅(t)) + \dot{q}₃(t)⋅sin(q₅(t))⋅c
⎢                                                                             
⎢-xCM⋅(\dot{q}₃(t)⋅sin(q₃(t))⋅sin(q₅(t)) - \dot{q}₃(t)⋅cos(q₃(t))⋅cos(q₄(t))⋅c
⎢                                                                             
⎣                                                                             

os(q₃(t)) + \dot{q}₄(t)⋅sin(q₄(t))⋅cos(q₃(t))⋅cos(q₅(t)) + \dot{q}₅(t)⋅sin(q₃(
                                                                              
os(q₅(t)) + \dot{q}₄(t)⋅sin(q₃(t))⋅sin(q₄(t))⋅cos(q₅(t)) + \dot{q}₅(t)⋅sin(q₃(
                                                                              
            r⋅\dot{q}₄(t)⋅cos(q₄(t)) - xCM⋅\dot{q}₄(t)⋅cos(q₄(t))⋅cos(q₅(t)) +

t))⋅cos(q₅(t)) + \dot{q}₅(t)⋅sin(q₅(t))⋅cos(q₃(t))⋅cos(q₄(t))) - zCM⋅\dot{q}₃(
                                                                              
t))⋅sin(q₅(t))⋅cos(q₄(t)) - \dot{q}₅(t)⋅cos(q₃(t))

In [ ]:
T = sympy.S.Half * cone['m'] * (CM_vel.dot(CM_vel))
V = cone['m'] * cone['g'] * CoM_w[2,0]
L = T-V
L = sympy.simplify(L)

In [ ]:
eoms = sympy.Matrix([
      L.diff(dq1).diff(t)-L.diff(q1),
      L.diff(dq2).diff(t)-L.diff(q2),
      L.diff(dq3).diff(t)-L.diff(q3),
      L.diff(dq4).diff(t)-L.diff(q4),
      L.diff(dq5).diff(t)-L.diff(q5)
])
eoms = eoms.subs(sub_rules)

### Rewrite Equations of Motion in Matrix Form

$$
M(q)\ddot{q} + h(q,\dot{q}) = 0
$$


In [ ]:
h = eoms.subs([(ddq1,0),(ddq2,0),(ddq3,0),(ddq4,0),(ddq5,0)])
Mddq = eoms-h
M = sympy.Matrix([linear_decomp(Mddq[i,0],ddq) for i in range(5)])

In [ ]:
M = sympy.simplify(M)
h = sympy.simplify(h)

### Nonholonomic Constrain (Rolling Constrain)

$\vec{v}_{A} = \vec{\omega} \times \vec{CA}$, where $C$ is contact point, $A$ is disk center. 

This is the mathematical representation of rolling without silping.

Angular velocity $\vec{\omega}$ can be calculated through its skew-symmetric matrix form, $\hat{\omega} = \dot{R}R^T$


In [ ]:
omega_hat = R_wb.diff(t).subs(sub_rules) * (R_wb.T)
omega_hat = sympy.simplify(omega_hat)
omega_hat

⎡                             0                                          -\dot
⎢                                                                             
⎢           \dot{q}₃(t) + \dot{q}₅(t)⋅cos(q₄(t))                              
⎢                                                                             
⎣-\dot{q}₄(t)⋅cos(q₃(t)) - \dot{q}₅(t)⋅sin(q₃(t))⋅sin(q₄(t))  -\dot{q}₄(t)⋅sin

{q}₃(t) - \dot{q}₅(t)⋅cos(q₄(t))             \dot{q}₄(t)⋅cos(q₃(t)) + \dot{q}₅
                                                                              
             0                               \dot{q}₄(t)⋅sin(q₃(t)) - \dot{q}₅
                                                                              
(q₃(t)) + \dot{q}₅(t)⋅sin(q₄(t))⋅cos(q₃(t))                              0    

(t)⋅sin(q₃(t))⋅sin(q₄(t))⎤
                         ⎥
(t)⋅sin(q₄(t))⋅cos(q₃(t))⎥
                         ⎥
                         ⎦

In [ ]:
omega = sympy.Matrix([ omega_hat[2,1], omega_hat[0,2], omega_hat[1,0] ])
omega

⎡-\dot{q}₄(t)⋅sin(q₃(t)) + \dot{q}₅(t)⋅sin(q₄(t))⋅cos(q₃(t))⎤
⎢                                                           ⎥
⎢\dot{q}₄(t)⋅cos(q₃(t)) + \dot{q}₅(t)⋅sin(q₃(t))⋅sin(q₄(t)) ⎥
⎢                                                           ⎥
⎣           \dot{q}₃(t) + \dot{q}₅(t)⋅cos(q₄(t))            ⎦

In [ ]:
r_CA = rot_z(psi) * rot_y(theta) * sympy.Matrix([ -cone['r'], 0 ,0 ])
r_CA

⎡-r⋅cos(q₃(t))⋅cos(q₄(t))⎤
⎢                        ⎥
⎢-r⋅sin(q₃(t))⋅cos(q₄(t))⎥
⎢                        ⎥
⎣      r⋅sin(q₄(t))      ⎦

In [ ]:
vel_A = omega.cross(r_CA)
vel_A = sympy.simplify(vel_A)
vel_A

⎡r⋅(\dot{q}₃(t)⋅sin(q₃(t))⋅cos(q₄(t)) + \dot{q}₄(t)⋅sin(q₄(t))⋅cos(q₃(t)) + \d
⎢                                                                             
⎢r⋅(-\dot{q}₃(t)⋅cos(q₃(t))⋅cos(q₄(t)) + \dot{q}₄(t)⋅sin(q₃(t))⋅sin(q₄(t)) - \
⎢                                                                             
⎣                                     r⋅\dot{q}₄(t)⋅cos(q₄(t))                

ot{q}₅(t)⋅sin(q₃(t))) ⎤
                      ⎥
dot{q}₅(t)⋅cos(q₃(t)))⎥
                      ⎥
                      ⎦

In [ ]:
rolling_constrains = [ dq1 - vel_A[0,0], dq2 - vel_A[1,0] ]
A_nh = sympy.Matrix([linear_decomp(eq,dq) for eq in rolling_constrains])
A_nh

⎡1  0  -r⋅sin(q₃(t))⋅cos(q₄(t))  -r⋅sin(q₄(t))⋅cos(q₃(t))  -r⋅sin(q₃(t))⎤
⎢                                                                       ⎥
⎣0  1  r⋅cos(q₃(t))⋅cos(q₄(t))   -r⋅sin(q₃(t))⋅sin(q₄(t))  r⋅cos(q₃(t)) ⎦

### Holonomic Constrain (Fixed Control Point)

$\dot{X}_{CP} = 0$

In [ ]:
CP_b = sympy.Matrix([ cone['zCP'], 0, cone['zCP'] ])
CP_b

⎡zCP⎤
⎢   ⎥
⎢ 0 ⎥
⎢   ⎥
⎣zCP⎦

In [ ]:
CP_w = R_wb * CP_b + T_wb
CP_w

⎡zCP⋅(-sin(q₃(t))⋅sin(q₅(t)) + cos(q₃(t))⋅cos(q₄(t))⋅cos(q₅(t))) + zCP⋅sin(q₄(
⎢                                                                             
⎢zCP⋅(sin(q₃(t))⋅cos(q₄(t))⋅cos(q₅(t)) + sin(q₅(t))⋅cos(q₃(t))) + zCP⋅sin(q₃(t
⎢                                                                             
⎣                     r⋅sin(q₄(t)) - zCP⋅sin(q₄(t))⋅cos(q₅(t)) + zCP⋅cos(q₄(t)

t))⋅cos(q₃(t)) + q₁(t)⎤
                      ⎥
))⋅sin(q₄(t)) + q₂(t) ⎥
                      ⎥
)                     ⎦

In [ ]:
CP_dot_w = CP_w.diff(t).subs(sub_rules)
CP_dot_w

⎡zCP⋅(-\dot{q}₃(t)⋅sin(q₃(t))⋅cos(q₄(t))⋅cos(q₅(t)) - \dot{q}₃(t)⋅sin(q₅(t))⋅c
⎢                                                                             
⎢zCP⋅(-\dot{q}₃(t)⋅sin(q₃(t))⋅sin(q₅(t)) + \dot{q}₃(t)⋅cos(q₃(t))⋅cos(q₄(t))⋅c
⎢                                                                             
⎣                                                                             

os(q₃(t)) - \dot{q}₄(t)⋅sin(q₄(t))⋅cos(q₃(t))⋅cos(q₅(t)) - \dot{q}₅(t)⋅sin(q₃(
                                                                              
os(q₅(t)) - \dot{q}₄(t)⋅sin(q₃(t))⋅sin(q₄(t))⋅cos(q₅(t)) - \dot{q}₅(t)⋅sin(q₃(
                                                                              
            r⋅\dot{q}₄(t)⋅cos(q₄(t)) - zCP⋅\dot{q}₄(t)⋅sin(q₄(t)) - zCP⋅\dot{q

t))⋅cos(q₅(t)) - \dot{q}₅(t)⋅sin(q₅(t))⋅cos(q₃(t))⋅cos(q₄(t))) - zCP⋅\dot{q}₃(
                                                                              
t))⋅sin(q₅(t))⋅cos(q₄(t)) + \dot{q}₅(t)⋅cos(q₃(t))

In [ ]:
A_cp = sympy.Matrix([linear_decomp(eq,dq) for eq in CP_dot_w[:2]])
A_cp

⎡1  0  zCP⋅(-sin(q₃(t))⋅cos(q₄(t))⋅cos(q₅(t)) - sin(q₅(t))⋅cos(q₃(t))) - zCP⋅s
⎢                                                                             
⎣0  1  zCP⋅(-sin(q₃(t))⋅sin(q₅(t)) + cos(q₃(t))⋅cos(q₄(t))⋅cos(q₅(t))) + zCP⋅s

in(q₃(t))⋅sin(q₄(t))  -zCP⋅sin(q₄(t))⋅cos(q₃(t))⋅cos(q₅(t)) + zCP⋅cos(q₃(t))⋅c
                                                                              
in(q₄(t))⋅cos(q₃(t))  -zCP⋅sin(q₃(t))⋅sin(q₄(t))⋅cos(q₅(t)) + zCP⋅sin(q₃(t))⋅c

os(q₄(t))  zCP⋅(-sin(q₃(t))⋅cos(q₅(t)) - sin(q₅(t))⋅cos(q₃(t))⋅cos(q₄(t)))⎤
                                                                          ⎥
os(q₄(t))  zCP⋅(-sin(q₃(t))⋅sin(q₅(t))⋅cos(q₄(t)) + cos(q₃(t))⋅cos(q₅(t)))⎦

### Paffian Constrain Matrix

$$
\mathbf{A}\dot{\mathbf{q}} = 0
$$

$$
\frac{d}{d t}(\mathbf{A}(\mathbf{q}) \dot{\mathbf{q}})=\mathbf{A}(\mathbf{q}) \ddot{\mathbf{q}}+\dot{\mathbf{A}}(\mathbf{q}) \dot{\mathbf{q}}=\mathbf{0}
$$

In [ ]:
A = sympy.Matrix([A_nh, A_cp])
A = sympy.simplify(A)
A

⎡1  0                                 -r⋅sin(q₃(t))⋅cos(q₄(t))                
⎢                                                                             
⎢0  1                                  r⋅cos(q₃(t))⋅cos(q₄(t))                
⎢                                                                             
⎢1  0  -zCP⋅(sin(q₃(t))⋅sin(q₄(t)) + sin(q₃(t))⋅cos(q₄(t))⋅cos(q₅(t)) + sin(q₅
⎢                                                                             
⎣0  1  zCP⋅(-sin(q₃(t))⋅sin(q₅(t)) + sin(q₄(t))⋅cos(q₃(t)) + cos(q₃(t))⋅cos(q₄

                                -r⋅sin(q₄(t))⋅cos(q₃(t))                      
                                                                              
                                -r⋅sin(q₃(t))⋅sin(q₄(t))                      
                                                                              
(t))⋅cos(q₃(t)))  zCP⋅(-sin(q₄(t))⋅cos(q₅(t)) + cos(q₄(t)))⋅cos(q₃(t))  -zCP⋅(
                                                   

In [ ]:
dAdt = A.diff(t).subs(sub_rules)
dAdt = sympy.simplify(dAdt)

### Equation of Motion


$\left[\begin{array}{cc}-\mathbf{A} & 0 \\ \mathbf{M} & -\mathbf{A}^{T}\end{array}\right]\left[\begin{array}{l}\ddot{\mathbf{q}} \\ \boldsymbol{\lambda}\end{array}\right]=\left[\begin{array}{c}\dot{\mathbf{A}} \dot{\mathbf{q}} \\ -\mathbf{h}\end{array}\right]$

In [ ]:
M_left = sympy.Matrix([[-A,sympy.zeros(4)],[M, -A.T]])
M_right = sympy.Matrix([[dAdt*sympy.Matrix(dq)],[-h]])

### Export the Dynamic Model

The dynamic model is represented as a linear equation:

$$
\mathbf{M}_{\mathbf{left}}(q) 
\begin{bmatrix}
\ddot{q} \\
\lambda
\end{bmatrix}
= \mathbf{M}_{\mathbf{right}}(q,\dot{q})
$$

Functions to calculate $\mathbf{M}_{\mathbf{left}}$ and $\mathbf{M}_{\mathbf{right}}$ are exported.

We want the output function from `sympy.lambdify` take $q$ and $\dot{q}$ as vectors, this is achieved using `sympy.MatrixSymbol`

In [ ]:
mq = sympy.MatrixSymbol('q',5,1)
mdq = sympy.MatrixSymbol('dq',5,1)

sub_rule_out = [(q1,mq[0,0]),(dq1,mdq[0,0]),
                (q2,mq[1,0]),(dq2,mdq[1,0]),
                (q3,mq[2,0]),(dq3,mdq[2,0]),
                (q4,mq[3,0]),(dq4,mdq[3,0]),
                (q5,mq[4,0]),(dq5,mdq[4,0])]

M_left = M_left.subs(sub_rule_out)
M_right = M_right.subs(sub_rule_out)

We can see that, `M_left` only depend on $q$, while `M_right` depends on $q$ and $\dot{q}$

In [ ]:
M_left.free_symbols, M_right.free_symbols

In [ ]:
f_M_left = sympy.lambdify([mq,cone['r'],cone['m'],cone['g'],cone['xCM'],cone['zCM'],cone['xCP'],cone['zCP']],M_left)
f_M_right = sympy.lambdify([mq,mdq,cone['r'],cone['m'],cone['g'],cone['xCM'],cone['zCM'],cone['xCP'],cone['zCP']],M_right)

In [ ]:
outputs = {
    'M_left': f_M_left,
    'M_right': f_M_right
}
import dill
dill.settings['recurse'] = True
dill.dump(outputs,open("rnw_model", "wb"))

### Use the Dynamic Model

In [ ]:
import numpy as np

# load the dynamic model from file
rnw_model = dill.load(open("rnw_model", "rb"))

# see what's inside the dynamic model
func2 = rnw_model['M_right']
??func2

# input the current state - q and qdot
fake_q = np.array([[0,0,0,np.pi/4,0]]).T
fake_dq = np.array([[1,1,1,1,1]]).T

# get the left and right matrix
eq_left = rnw_model['M_left'](fake_q,1,1,9.8,1,1,2,3)
eq_right = rnw_model['M_right'](fake_q,fake_dq,1,1,9.8,1,1,2,3)

# solve the linear system, get q_dot_dot and lagarange multipliers
sol = np.linalg.solve(eq_left,eq_right)
sol_ddq = sol[:5].T
sol_multipliers = sol[5:].T
sol_ddq, sol_multipliers

(array([[ 1.66066017e+01,  1.25607397e-15, -2.00000000e+00,
          2.00710678e+01,  2.82842712e+00]]),
 array([[-4.39238816e+00,  7.58107702e-16,  1.54634560e+01,
         -0.00000000e+00]]))